# RFM ANALYSIS

RFM Analysis is a technique used to understand customer behavior and how valuable the customer is to a business. The essential parts of this analysis include Recency, Frequency, and Monetary value. RFM analysis helps business owners to optimize customer engagement.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"

data = pd.read_csv("C:\\Users\\HP\Desktop\\rfm_data.csv")

In [2]:
print(data.head())

   CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0        8814   2023-04-11             943.31          Product C   890075   
1        2188   2023-04-11             463.70          Product A   176819   
2        4608   2023-04-11              80.28          Product A   340062   
3        2559   2023-04-11             221.29          Product A   239145   
4        9482   2023-04-11             739.56          Product A   194545   

   Location  
0     Tokyo  
1    London  
2  New York  
3    London  
4     Paris  


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


In [4]:
from datetime import datetime
data['PurchaseDate']=pd.to_datetime(data['PurchaseDate'])

To calculate recency, find the diffence between present date and purchase date.The code below calculates how many days ago the customer made their last purchase.

In [5]:
#creating the recency column
data['Recency']=(datetime.now().date()-data['PurchaseDate'].dt.date).dt.days

To calculate the frequency of purchase, the transations are grouped by customerID and the number of orders for each group is counted. All in a new dataframe. After this is done, the created df is merged with the original dataset. 

In [6]:
Frequency_df = data.groupby('CustomerID')['OrderID'].count().reset_index()
Frequency_df.head()

,CustomerID,OrderID
0,1011,2
1,1025,1
2,1029,1
3,1046,1
4,1049,1


In [7]:
Frequency_df.rename(columns={'OrderID': 'Frequency'}, inplace=True)
data=data.merge(Frequency_df, on='CustomerID', how='left')

To calculate the total monetary value for each customer, the data is grouped by customerID and the transaction amount summed for each group.

In [8]:
Monetary_df=data.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
Monetary_df.rename(columns={'TransactionAmount': 'MonetaryValue'}, inplace=True)
data=data.merge(Monetary_df, on='CustomerID', how='left')

In [9]:
data.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,123,1,943.31
1,2188,2023-04-11,463.70,Product A,176819,London,123,1,463.70
2,4608,2023-04-11,80.28,Product A,340062,New York,123,1,80.28
3,2559,2023-04-11,221.29,Product A,239145,London,123,1,221.29
4,9482,2023-04-11,739.56,Product A,194545,Paris,123,1,739.56


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   int64         
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   int64         
 5   Location            1000 non-null   object        
 6   Recency             1000 non-null   int64         
 7   Frequency           1000 non-null   int64         
 8   MonetaryValue       1000 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 78.1+ KB


# RFM SCORES

To calculate RFM scores, the scoring criteria is first determined. Then based on the RFM values previously calculated and the scoring criteria, scores are assigned to corresponding values.

In [11]:
#scoring criteria for each component
R_scores=[5,4,3,2,1]
F_scores=[1,2,3,4,5]
M_scores=[1,2,3,4,5]
#calculating RFM scores
data['Recency Score']= pd.cut(data['Recency'],bins=5,labels=R_scores)
data['Frequency Score']= pd.cut(data['Frequency'],bins=5,labels=F_scores)
data['MonetaryValue Score']= pd.cut(data['MonetaryValue'],bins=5,labels=M_scores)

In [12]:
data.head(15)

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,Recency Score,Frequency Score,MonetaryValue Score
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,123,1,943.31,1,1,2
1,2188,2023-04-11,463.70,Product A,176819,London,123,1,463.70,1,1,1
2,4608,2023-04-11,80.28,Product A,340062,New York,123,1,80.28,1,1,1
3,2559,2023-04-11,221.29,Product A,239145,London,123,1,221.29,1,1,1
4,9482,2023-04-11,739.56,Product A,194545,Paris,123,1,739.56,1,1,2
5,8483,2023-04-11,375.23,Product C,691194,Paris,123,1,375.23,1,1,1
6,8317,2023-04-11,272.56,Product B,826847,New York,123,2,974.88,1,3,3
7,6911,2023-04-11,433.33,Product C,963918,Tokyo,123,1,433.33,1,1,1
8,8993,2023-04-12,16.55,Product D,112426,New York,122,1,16.55,1,1,1
9,3519,2023-04-12,464.63,Product C,139726,New York,122,1,464.63,1,1,1


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CustomerID           1000 non-null   int64         
 1   PurchaseDate         1000 non-null   datetime64[ns]
 2   TransactionAmount    1000 non-null   float64       
 3   ProductInformation   1000 non-null   object        
 4   OrderID              1000 non-null   int64         
 5   Location             1000 non-null   object        
 6   Recency              1000 non-null   int64         
 7   Frequency            1000 non-null   int64         
 8   MonetaryValue        1000 non-null   float64       
 9   Recency Score        1000 non-null   category      
 10  Frequency Score      1000 non-null   category      
 11  MonetaryValue Score  1000 non-null   category      
dtypes: category(3), datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 81.7+ KB


In [14]:
#changing the data types from object to integer
data['Recency Score']=data['Recency Score'].astype(int)
data['Frequency Score']=data['Frequency Score'].astype(int)
data['MonetaryValue Score']=data['MonetaryValue Score'].astype(int)

In [15]:
#calculating the RFM score for each customer
data['RFM Score']= data['Recency Score'] + data['Frequency Score'] + data['MonetaryValue Score']

In [17]:
#creating customer category and value segments based on RFM Scores
Value_Segment = ['Low_value','Mid_value','High_Value']
data['ValueSegment']=pd.qcut(data['RFM Score'], q=3, labels = Value_Segment)

data['CustomerCategory'] = '' #this creates an empty stringtype column
data.loc[data['RFM Score']>=13,'CustomerCategory']= 'Champions'
data.loc[(data['RFM Score']>=11) & (data['RFM Score']<13), 'CustomerCategory']= 'Loyal Customers'
data.loc[(data['RFM Score']>=9) & (data['RFM Score']<11), 'CustomerCategory']= 'Potential Loyal Customers'
data.loc[(data['RFM Score']>=7) & (data['RFM Score']<9), 'CustomerCategory']= 'Engaged Customers'
data.loc[(data['RFM Score']>=5) & (data['RFM Score']<7), 'CustomerCategory']= 'At Risk'
data.loc[(data['RFM Score']>=3) & (data['RFM Score']<5), 'CustomerCategory']= 'Lost'
data.head(15)

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,Recency Score,Frequency Score,MonetaryValue Score,RFM Score,ValueSegment,CustomerCategory
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,123,1,943.31,1,1,2,4,Low_value,Lost
1,2188,2023-04-11,463.70,Product A,176819,London,123,1,463.70,1,1,1,3,Low_value,Lost
2,4608,2023-04-11,80.28,Product A,340062,New York,123,1,80.28,1,1,1,3,Low_value,Lost
3,2559,2023-04-11,221.29,Product A,239145,London,123,1,221.29,1,1,1,3,Low_value,Lost
4,9482,2023-04-11,739.56,Product A,194545,Paris,123,1,739.56,1,1,2,4,Low_value,Lost
5,8483,2023-04-11,375.23,Product C,691194,Paris,123,1,375.23,1,1,1,3,Low_value,Lost
6,8317,2023-04-11,272.56,Product B,826847,New York,123,2,974.88,1,3,3,7,Mid_value,Engaged Customers
7,6911,2023-04-11,433.33,Product C,963918,Tokyo,123,1,433.33,1,1,1,3,Low_value,Lost
8,8993,2023-04-12,16.55,Product D,112426,New York,122,1,16.55,1,1,1,3,Low_value,Lost
9,3519,2023-04-12,464.63,Product C,139726,New York,122,1,464.63,1,1,1,3,Low_value,Lost


In [18]:
#exporting data for visualization
data.to_csv('c:\\Users\\HP\\Desktop\\RFM_Analysis.csv')